In [1]:
import sys
sys.path.append('../code')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from sklearn.model_selection import ShuffleSplit
import pickle

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0")
#device = torch.device('cpu')

torch.backends.cudnn.benchmark = True

import contrastive_functions

In [2]:
data_dict = contrastive_functions.get_marker_decode_dataframes()
wrist_df = data_dict['wrist_df']
task_neural_df = data_dict['task_neural_df']
notask_neural_df = data_dict['notask_neural_df']
metadata = data_dict['metadata']

In [3]:
trial_ids = task_neural_df['trial'].unique()
num_trials_filtered = len(trial_ids)

#Generate cv_dict for regular train/test/validate split
cv_split = ShuffleSplit(n_splits=5, test_size=.25, random_state=3)
val_split = ShuffleSplit(n_splits=1, test_size=.25, random_state=3)
cv_dict = {}
for fold, (train_val_idx, test_idx) in enumerate(cv_split.split(trial_ids)):
    for t_idx, v_idx in val_split.split(train_val_idx): #No looping, just used to split train/validation sets
        cv_dict[fold] = {'train_idx':trial_ids[train_val_idx[t_idx]], 
                        'test_idx':trial_ids[test_idx], 
                        'validation_idx':trial_ids[train_val_idx[v_idx]]} 

In [4]:
neural_offset = 5 # try 50-150 ms offset
window_size = 70
label_col = 'layout'
func_dict = {'wiener': contrastive_functions.run_wiener, 'rnn': contrastive_functions.run_rnn}

fpath = '../data/SPK20220308/neuron_num_results/'

num_repeats = 10
# num_neuron_list = np.arange(2,51,4)
num_neuron_list = [2,4,6,8,10]


num_neuron_results_dict = {'num_neuron_list': num_neuron_list}
for repeat_idx in range(num_repeats):
    rng = np.random.default_rng(repeat_idx) # new set of shuffled neurons seeded by repeat_idx
    random_units = rng.choice(range(85), size=85).astype(str)

    num_neuron_results_dict[f'repeat_{repeat_idx}'] = {'random_units': random_units}
    for num_neurons in num_neuron_list:

        # Filter neural_df with task info to random subset of neurons
        task_unit_mask = np.in1d(task_neural_df['unit'].values, random_units[:num_neurons])
        layout_mask = task_neural_df['unit'].str.contains(pat='layout')

        task_neural_df_filtered = task_neural_df[np.logical_or.reduce([task_unit_mask, layout_mask])].reset_index(drop=True)

        # Filter neural_df without task info to random subset of neurons
        notask_unit_mask = np.in1d(notask_neural_df['unit'].values, random_units[:num_neurons])
        notask_neural_df_filtered = notask_neural_df[np.logical_or.reduce([notask_unit_mask])].reset_index(drop=True)

        df_dict = {'task': {'df': task_neural_df_filtered, 'task_info': True, 'num_cat': 4}, # num_cat = number of categorical features
                   'notask': {'df': notask_neural_df_filtered, 'task_info': False, 'num_cat': 0}}
        

        decode_results = dict()
        for func_name, func in func_dict.items():
            decode_results[func_name] = dict()
            for df_type, pred_df in df_dict.items():
                model, res_dict = func(wrist_df, pred_df['df'], neural_offset, cv_dict, metadata, task_info=pred_df['task_info'],
                                       window_size=window_size, num_cat=pred_df['num_cat'], label_col=label_col)

                decode_results[func_name][df_type] = res_dict

                # Save results on every loop in case early stop
                num_neuron_results_dict[f'repeat_{repeat_idx}'][f'num_neuron_{num_neurons}'] = decode_results
                #Save metadata
                output = open(f'{fpath}num_neuron_results.pkl', 'wb')
                pickle.dump(num_neuron_results_dict, output)
                output.close()

                if func_name == 'rnn':
                    torch.save(model.state_dict(), f'{fpath}models/{df_type}_neurons{num_neurons}_repeat{repeat_idx}.pt')





******.***
Epoch: 10/1000 ... Train Loss: 1.9461  ... Validation Loss: 1.8954
******.***
Epoch: 20/1000 ... Train Loss: 1.5307  ... Validation Loss: 1.4622
*.*******.
Epoch: 30/1000 ... Train Loss: 1.4440  ... Validation Loss: 1.3776
*.*******.
Epoch: 40/1000 ... Train Loss: 1.3656  ... Validation Loss: 1.3128
*.*.*...*.
Epoch: 50/1000 ... Train Loss: 1.3226  ... Validation Loss: 1.2824
*.*...*.*.
Epoch: 60/1000 ... Train Loss: 1.2954  ... Validation Loss: 1.2527
..... Early Stop; Min Epoch: 59
**********
Epoch: 10/1000 ... Train Loss: 0.8594  ... Validation Loss: 0.8056
**********
Epoch: 20/1000 ... Train Loss: 0.6913  ... Validation Loss: 0.6689
********.*
Epoch: 30/1000 ... Train Loss: 0.3846  ... Validation Loss: 0.3639
***..*..*.
Epoch: 40/1000 ... Train Loss: 0.2746  ... Validation Loss: 0.2949
*.*...... Early Stop; Min Epoch: 43
**********
Epoch: 10/1000 ... Train Loss: 1.9248  ... Validation Loss: 1.8714
**********
Epoch: 20/1000 ... Train Loss: 1.5210  ... Validation Loss: 1.4